# import libraries

In [2]:
import pandas as pd
import numpy as np
import re

# load data

In [3]:
movies_df = pd.read_csv("ml-latest-small/movies.csv")
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [4]:
ratings_df = pd.read_csv("ml-latest-small/ratings.csv")
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


# Data cleaning and prepration

In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


### extract date from tittle

In [6]:
def extract_date(text):
    pattern = r'\(([^)]*)\)[^()]*$'
    matches = re.findall(pattern, text)
    if len(matches):
        return matches[0][:4]
    else:
        return np.nan

In [7]:
movies_df["movie_date"] = movies_df["title"].apply(extract_date)
movies_df["movie_date"] = pd.to_datetime(movies_df["movie_date"]).dt.year

In [8]:
movies_df["movie_date"].dtype

dtype('float64')

In [9]:
movies_df["genres"] = movies_df["genres"].apply(lambda x : str(x).split(sep='|'))

### merge two data frame and convert timestamp to datatime

In [10]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [11]:
final_data = pd.merge(movies_df, ratings_df, on="movieId")

In [12]:
final_data.columns

Index(['movieId', 'title', 'genres', 'movie_date', 'userId', 'rating',
       'timestamp'],
      dtype='object')

In [13]:
final_data["timestamp"] = pd.to_datetime(final_data['timestamp'], unit='s')

In [14]:
final_data["transaction_month"] = final_data["timestamp"].dt.month_name()

In [15]:
final_data.head()

,movieId,title,genres,movie_date,userId,rating,timestamp,transaction_month
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,1,4.0,2000-07-30 18:45:03,July
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,5,4.0,1996-11-08 06:36:02,November
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,7,4.5,2005-01-25 06:52:26,January
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,15,2.5,2017-11-13 12:59:30,November
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,17,4.5,2011-05-18 05:28:03,May


In [16]:
final_data["movie_date"].min()

1902.0

In [17]:
final_data["movie_date"].max()

2018.0

In [18]:
bins = list(range(1900,2030,10))
labels = [str(bin) + "s" for bin in bins]

In [19]:
# Use pd.cut() to bin the years column
final_data['movie_date_bin'] = pd.cut(final_data["movie_date"], bins,labels=labels[:-1])

In [20]:
final_data.head()

,movieId,title,genres,movie_date,userId,rating,timestamp,transaction_month,movie_date_bin
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,1,4.0,2000-07-30 18:45:03,July,1990s
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,5,4.0,1996-11-08 06:36:02,November,1990s
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,7,4.5,2005-01-25 06:52:26,January,1990s
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,15,2.5,2017-11-13 12:59:30,November,1990s
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,17,4.5,2011-05-18 05:28:03,May,1990s


### create cloumn represent order of transaction from time stamp

In [21]:
final_data['transaction_from_movie_year'] = final_data["timestamp"].dt.year - final_data["movie_date"]

In [22]:
final_data.head()

,movieId,title,genres,movie_date,userId,rating,timestamp,transaction_month,movie_date_bin,transaction_from_movie_year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,1,4.0,2000-07-30 18:45:03,July,1990s,5.0
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,5,4.0,1996-11-08 06:36:02,November,1990s,1.0
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,7,4.5,2005-01-25 06:52:26,January,1990s,10.0
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,15,2.5,2017-11-13 12:59:30,November,1990s,22.0
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,17,4.5,2011-05-18 05:28:03,May,1990s,16.0


# get last transaction movie history dataframe

In [23]:
# Sort the dataframe by timestamp in ascending order
df_copy_sorted = final_data.sort_values(by='timestamp', ascending=True)

# Group the dataframe by user ID and shift the 'movieId' column by one row
def shift_movie_id(group):
    group['prev_movieId'] = group['movieId'].shift(1)
    return group

df_copy_sorted = df_copy_sorted.groupby('userId').apply(shift_movie_id)

/tmp/ipykernel_35041/1319723476.py:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_copy_sorted = df_copy_sorted.groupby('userId').apply(shift_movie_id)


In [24]:
df_copy_sorted.head()

,movieId,title,genres,movie_date,userId,rating,timestamp,transaction_month,movie_date_bin,transaction_from_movie_year,prev_movieId
15993,590,Dances with Wolves (1990),"[Adventure, Drama, Western]",1990.0,429,5.0,1996-03-29 18:36:55,March,1980s,6.0,NaN
5936,222,Circle of Friends (1995),"[Drama, Romance]",1995.0,429,4.0,1996-03-29 18:36:55,March,1990s,1.0,590.0
12093,434,Cliffhanger (1993),"[Action, Adventure, Thriller]",1993.0,429,4.0,1996-03-29 18:36:55,March,1990s,3.0,222.0
16167,592,Batman (1989),"[Action, Crime, Thriller]",1989.0,429,5.0,1996-03-29 18:36:55,March,1980s,7.0,434.0
6119,225,Disclosure (1994),"[Drama, Thriller]",1994.0,429,4.0,1996-03-29 18:36:55,March,1990s,2.0,592.0


# final dataset

In [25]:
data_used = df_copy_sorted[["movieId", "userId", "genres", "transaction_month", "movie_date_bin", "timestamp","transaction_from_movie_year","prev_movieId", "rating"]]

# Data pre-processing

In [26]:
data_used.head()

,movieId,userId,genres,transaction_month,movie_date_bin,timestamp,transaction_from_movie_year,prev_movieId,rating
15993,590,429,"[Adventure, Drama, Western]",March,1980s,1996-03-29 18:36:55,6.0,NaN,5.0
5936,222,429,"[Drama, Romance]",March,1990s,1996-03-29 18:36:55,1.0,590.0,4.0
12093,434,429,"[Action, Adventure, Thriller]",March,1990s,1996-03-29 18:36:55,3.0,222.0,4.0
16167,592,429,"[Action, Crime, Thriller]",March,1980s,1996-03-29 18:36:55,7.0,434.0,5.0
6119,225,429,"[Drama, Thriller]",March,1990s,1996-03-29 18:36:55,2.0,592.0,4.0


### 1 - Create Aggregated History Matrix

In [27]:
# Replace None with the appropriate values
agg_history = pd.pivot_table(data_used,
               values="rating", index="userId", columns="movieId", aggfunc=lambda x: 0 if pd.isnull(x).all() else 1, fill_value=0)
agg_history

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
607,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
agg_history_norm = agg_history / agg_history.values.sum(axis=1, keepdims=True)
agg_history_norm

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.004310,0.000000,0.004310,0.0,0.0,0.004310,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.022727,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.000897,0.000000,0.000000,0.0,0.0,0.000000,0.000897,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.005348,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.001203,0.001203,0.001203,0.0,0.0,0.000000,0.000000,0.0,0.0,0.001203,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
len(data_used["movieId"].unique()), len(data_used["userId"].unique())

(9724, 610)

In [30]:
agg_history.shape

(610, 9724)

### get one hot encoding for catogorical data 

In [31]:
data_used.columns

Index(['movieId', 'userId', 'genres', 'transaction_month', 'movie_date_bin',
       'timestamp', 'transaction_from_movie_year', 'prev_movieId', 'rating'],
      dtype='object')

In [32]:
# Use pd.get_dummies() to convert the column of lists into dummy variables
dummies_genres = pd.get_dummies(data_used['genres'].apply(pd.Series).stack()).groupby(level=0).sum()

In [33]:
dummies_the_rest = pd.get_dummies(data_used[["transaction_month","movie_date_bin"]])

In [34]:
dummies_the_rest

,transaction_month_April,transaction_month_August,transaction_month_December,transaction_month_February,transaction_month_January,transaction_month_July,transaction_month_June,transaction_month_March,transaction_month_May,transaction_month_November,...,movie_date_bin_1920s,movie_date_bin_1930s,movie_date_bin_1940s,movie_date_bin_1950s,movie_date_bin_1960s,movie_date_bin_1970s,movie_date_bin_1980s,movie_date_bin_1990s,movie_date_bin_2000s,movie_date_bin_2010s
15993,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
5936,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
12093,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
16167,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
6119,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
100800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
66124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
66116,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [35]:
cats_ohe = pd.concat([dummies_genres, dummies_the_rest],axis=1)

In [36]:
cats_ohe

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,movie_date_bin_1920s,movie_date_bin_1930s,movie_date_bin_1940s,movie_date_bin_1950s,movie_date_bin_1960s,movie_date_bin_1970s,movie_date_bin_1980s,movie_date_bin_1990s,movie_date_bin_2000s,movie_date_bin_2010s
0,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0,0,1,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,0,1,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
100832,0,0,0,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
100833,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
100834,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### add train column give latest transaction to test data

In [37]:
data_used["train"] = (data_used.groupby('userId')[['timestamp']].transform(lambda x: x.where(x != x.max(), 0).apply(lambda y: 1 if y != 0 else 0)))

/tmp/ipykernel_35041/3941002266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used["train"] = (data_used.groupby('userId')[['timestamp']].transform(lambda x: x.where(x != x.max(), 0).apply(lambda y: 1 if y != 0 else 0)))


In [38]:
data_used.head()

,movieId,userId,genres,transaction_month,movie_date_bin,timestamp,transaction_from_movie_year,prev_movieId,rating,train
15993,590,429,"[Adventure, Drama, Western]",March,1980s,1996-03-29 18:36:55,6.0,NaN,5.0,1
5936,222,429,"[Drama, Romance]",March,1990s,1996-03-29 18:36:55,1.0,590.0,4.0,1
12093,434,429,"[Action, Adventure, Thriller]",March,1990s,1996-03-29 18:36:55,3.0,222.0,4.0,1
16167,592,429,"[Action, Crime, Thriller]",March,1980s,1996-03-29 18:36:55,7.0,434.0,5.0,1
6119,225,429,"[Drama, Thriller]",March,1990s,1996-03-29 18:36:55,2.0,592.0,4.0,1


### gether all train data

In [39]:
new_df = pd.concat([data_used[["movieId", "userId"]],cats_ohe, data_used[["transaction_from_movie_year","prev_movieId","train", "rating"]]], axis=1)
new_df

,movieId,userId,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,movie_date_bin_1960s,movie_date_bin_1970s,movie_date_bin_1980s,movie_date_bin_1990s,movie_date_bin_2000s,movie_date_bin_2010s,transaction_from_movie_year,prev_movieId,train,rating
15993,590,429,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,6.0,NaN,1,5.0
5936,222,429,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1.0,590.0,1,4.0
12093,434,429,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,3.0,222.0,1,4.0
16167,592,429,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,7.0,434.0,1,5.0
6119,225,429,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,2.0,592.0,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100779,187031,514,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0.0,187593.0,1,2.5
100800,187595,514,0,1,1,0,1,0,0,0,...,0,0,0,0,0,1,0.0,187031.0,1,3.0
66124,5247,514,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,41.0,187595.0,1,2.5
66116,5246,514,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,38.0,5247.0,1,1.5


### split train and test 

In [40]:
train = new_df[new_df["train"] == 1]
test = new_df[new_df["train"] == 0]

In [41]:
train.columns

Index(['movieId', 'userId', '(no genres listed)', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'transaction_month_April', 'transaction_month_August',
       'transaction_month_December', 'transaction_month_February',
       'transaction_month_January', 'transaction_month_July',
       'transaction_month_June', 'transaction_month_March',
       'transaction_month_May', 'transaction_month_November',
       'transaction_month_October', 'transaction_month_September',
       'movie_date_bin_1900s', 'movie_date_bin_1910s', 'movie_date_bin_1920s',
       'movie_date_bin_1930s', 'movie_date_bin_1940s', 'movie_date_bin_1950s',
       'movie_date_bin_1960s', 'movie_date_bin_1970s', 'movie_date_bin_1980s',
       'movie_date_bin_1990s', 'movie_date_bin_2000s', 'movie_date_bin_2010s',
       'transaction_from_mo

In [42]:
print('The dimensions of the training subset: ', train.shape[0], 'X', train.shape[1])
print('The dimensions of the test subset: ', test.shape[0], 'X', test.shape[1])

The dimensions of the training subset:  99947 X 50
The dimensions of the test subset:  889 X 50


# save final data

In [43]:
train.to_csv("final_data/train")

In [44]:
test.to_csv("final_data/test")

In [45]:
agg_history_norm.to_csv("final_data/agg_history_norm")

In [46]:
all_data = new_df.to_csv("final_data/all_data")

In [47]:
new_df.loc[0]

movieId                           1.0
userId                            1.0
(no genres listed)                0.0
Action                            0.0
Adventure                         1.0
Animation                         1.0
Children                          1.0
Comedy                            1.0
Crime                             0.0
Documentary                       0.0
Drama                             0.0
Fantasy                           1.0
Film-Noir                         0.0
Horror                            0.0
IMAX                              0.0
Musical                           0.0
Mystery                           0.0
Romance                           0.0
Sci-Fi                            0.0
Thriller                          0.0
War                               0.0
Western                           0.0
transaction_month_April           0.0
transaction_month_August          0.0
transaction_month_December        0.0
transaction_month_February        0.0
transaction_